<a href="https://colab.research.google.com/github/gnitnaw/Programs/blob/master/Chapter3/SCO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python --version

Python 3.7.11


# Example 3.5
## Import modules

In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt
print(np.__version__)

1.19.5


## Set random seed

In [3]:
np.random.seed(120)

# Example 3.17
## Input

In [4]:
c = np.array([ -6.089, -17.164, -34.054,  -5.914, -24.721,
              -14.986, -24.100, -10.708, -26.662, -22.179])

nDim = 10
#xStart = np.array([0.04067247,0.14765159,0.78323637,0.00141368,0.48526222,
#                   0.00069291,0.02736897,0.01794290,0.03729653,0.09685870])

tree = []

def addBranch(x):
  tree.append(x)

def S(x):
  y = 0
  xSum = np.sum(x)
  for i in range(nDim):
    y = y + x[i]*(c[i]+math.log(x[i]/xSum))
  return y

def constraint_1(x): #x[0]
  return 2-2*x[1]-2*x[2]-x[5]-x[9]

def constraint_2(x): #x[3]
  return 1-2*x[4]-x[5]-x[6]

def constraint_3(x): #x[2]
  return 1-x[6]-x[7]-2*x[8]-x[9]

def constraint_4(x):
  for i in range(nDim):
    if x[i] < 0.000001:
      return False
  return True

def dimension_reduce(x):
  y = x.copy()
  y[0] = constraint_1(x)
  y[3] = constraint_2(x)
  y[2] = constraint_3(x)
  return y

def penalty(x):
  y = 0
  for i in range(nDim):
    y = y + max(-(x[i]-0.000001), 0)
  y = y + S(x)
  return y

def generate_Start() :
  xStart = dimension_reduce(np.random.rand(nDim))
  while constraint_4(xStart) == False:
    xStart = dimension_reduce(np.random.rand(nDim))
  return xStart

def sampling_level1(n) : 
  x = np.zeros((n, nDim))
  for i in range(n):
    x[i] = generate_Start()
  return x

def sampling_level2(n, initValue):
  x = np.random.random((n, nDim))*0.1 + initValue
  return x

N = 100
rho = 0.5
omega = 2.0
debugMode = False
MaxTry = 500
tMax = 300
#x = xStart
#print(x)
#print(penalty(x))


## Step 1 : Generate X1...XN via uniform sampling. Set t = 0 and N_e = ceil(N rho)

In [5]:
#XGet = np.array([0.75379437, 0.48226277, 0.55370205, 0.01726460, 0.29904536, 
#                 0.17542843, 0.20921625, 0.06626207, 0.07727083, 0.01627797])
XGet = np.array([0.04067247,0.14765159,0.78323637,0.00141368,0.48526222,
                   0.00069291,0.02736897,0.01794290,0.03729653,0.09685870])
print(S(XGet))

XGet = np.array([1.77540106e+00, 8.95871298e-02, 9.49466098e-01, 5.54672599e-02, 4.37438025e-01,
                 2.80431617e-02, 4.16135281e-02, 2.47127879e-04, 3.85182508e-03, 9.69596179e-04])
print(S(XGet))

Xt = sampling_level1(N)
#Xt = sampling_level2(N, 0.1)
t = 0
N_elite = math.ceil(N*rho)
stopLoop = False
if debugMode :
  print(Xt[0:10])

SX = np.empty(N)
for i in range(N):
  SX[i] = penalty(Xt[i])

eliteXindex = np.argsort(SX)

-47.761090811433775
-61.52523707787246


## 1st while loop (Step 2)

In [6]:
while stopLoop == False :
  
  SX = SX[eliteXindex]
  Xt = Xt[eliteXindex]
  
  # Store elite X

  # Refresh S_best and X_best (Step 3)
  S_best = SX[0]
  X_best = Xt[0]

  # Step 4 : Draw Bi
  B_elite = np.random.binomial(1, 0.5, N_elite)

  ## 2nd loop, 1st for loop (Step. 5)
  RS = math.ceil(N*1.0/N_elite)
  for i in range(N_elite):
    R = B_elite[i] + RS  ## Step 6.
    if R > N_elite :
      R = N_elite
    Y = Xt[i].copy()  ## Step 7
    
    for j in range(R) : ## Step 8
      Y_p = Y.copy()
      I = np.random.randint(N_elite, size=N_elite) ## Step 9
      for k in range(N_elite):
        while I[k] == i :
          I[k] = np.random.randint(N_elite, size=1).item()
      if debugMode :
        print("i = ", i, " I[j] = ", I[j])
      SigmaX = omega*np.linalg.norm(Xt[i]-Xt[I[j]])
      Pi = np.arange(nDim)
      np.random.shuffle(Pi)  ## Step 10
      if debugMode :
        print(i, ": ", j, ": ", I, " :", SigmaX)
      for k in range(nDim): ## Step 11
        for Try in range(MaxTry):  ## Step 12
          if Pi[k] != 0 and Pi[k] != 2 and Pi[k] != 3 :
            Y_pp = Y_p.copy()
            Y_pp[Pi[k]] = Y_p[Pi[k]] + SigmaX*np.random.randn()  ## Step 13
            Y_pp = dimension_reduce(Y_pp)
            if Y_pp[Pi[k]] > 10**(-6) and Y_pp[0] > 10**(-6) and Y_pp[2] > 10**(-6) and Y_pp[3] > 10**(-6) :
              Y_p = Y_pp.copy()
              PPY = penalty(Y_p)
              if PPY < penalty(Y) :  ## Step 14
                Y = Y_p.copy()
                addBranch([t, i, j, k, Y, PPY])
                break
        Xt[i] = Y.copy()    ## Step 15
        SX[i] = penalty(Xt[i])
  eliteXindex = np.argsort(SX)
  t = t+1
  if (t%10 == 0) : 
    print("t = ", t, " Best S = ", S_best, " X = ", X_best)
  
  if t>=tMax :
    stopLoop = True


print("=========================================")
print(tree)

if debugMode :
  print(SX)
  print(eliteXindex)
  print(B_elite)

t =  10  Best S =  -60.691183992186424  X =  [1.95268153e+00 2.11233532e-03 9.50763913e-01 1.14145455e-01
 4.31149344e-01 1.49752086e-02 8.58064735e-03 2.71092485e-02
 5.68166557e-04 1.24098580e-02]
t =  20  Best S =  -60.77826605374272  X =  [1.95787108e+00 2.11233532e-03 9.53420719e-01 1.14145455e-01
 4.31149344e-01 1.49752086e-02 8.58064735e-03 2.44524426e-02
 5.68166557e-04 1.24098580e-02]
t =  30  Best S =  -60.946103012201526  X =  [1.93188884e+00 2.38176257e-02 9.81284259e-01 2.15459986e-01
 3.89978941e-01 3.47982757e-03 1.10230417e-03 2.60239065e-03
 7.20807456e-03 5.94897387e-04]
t =  40  Best S =  -61.2472145323613  X =  [1.82956980e+00 4.95310554e-02 9.37961268e-01 1.44177851e-02
 4.62278895e-01 3.47455500e-02 2.62788748e-02 1.25799905e-02
 8.98182113e-04 2.13835025e-02]
t =  50  Best S =  -61.37691604018519  X =  [1.88504739e+00 2.98660113e-02 9.59069469e-01 6.44728324e-04
 4.78294792e-01 2.08551610e-02 2.19105273e-02 2.30189268e-03
 4.03455863e-03 8.64899394e-03]
t =  60  